<a href="https://colab.research.google.com/github/RGVarao/Teste01/blob/main/Teste01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -U -q google-generativeai #intalando o Python SDK (Kit para desenvolvimento de programas)

In [8]:
#Importações e configurações iniciais
import numpy as np # A biblioteca NumPy faz cálculos matemáticos, calcula vetores, etc.
import pandas as pd # Trabalha com tabelas e células como no Google Cheets e Excel
import google.generativeai as genai # Carrega os modelos de inteligência generativa

GOOGLE_API_KEY = " colocar a API key aqui" # Minha API de identificação
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
for m in genai.list_models(): # Verificar quais modelos trabalham com Embedding
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [13]:
# Exemplo de embadding
title = "A próxima geraçao de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geraçao de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API e Google IA Studio: uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.056239724, -0.028828401, -0.028407065, 0.021560036, 0.0681524, -0.0057308455, -0.036269575, -0.023684472, 0.060241852, 0.057910744, 0.013408415, 0.014389568, -0.057097834, -0.024733262, 0.00898069, -0.0077210036, 0.025170403, -0.021847837, -0.027427804, -0.0056608436, 0.0034079587, 0.016396975, -0.039816514, -0.06783299, -0.013078442, 0.027342282, 0.0073865927, -0.037767973, -0.034479335, 0.028858822, -0.0526408, 0.046821136, -0.034029126, 0.02160852, -0.04380999, -0.029885687, -0.029850082, -0.04698979, -0.0013939026, 0.004568283, 0.00529423, -0.08200831, -0.0154455, 0.030490372, -0.0028926516, -0.025875403, 0.050136596, 0.0503554, 0.022819923, -0.05751762, 0.020760523, 0.024692863, 0.05847709, -0.04650604, -0.0053605684, -0.0073937345, 0.021011218, -0.037831202, 0.020847434, -0.0066659143, 0.006718179, 0.025740499, -0.00403686, 0.055300757, 0.031212818, -0.06497432, -0.037558846, -0.0021994603, 0.004787631, 0.048972018, 0.002591265, 0.013234167, 0.044714656, -0.06509

In [14]:
# Listagem dos documentos que serão buscados

DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [25]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [18]:
model = "models/embedding-001"

In [22]:
def embed_fn(title, text): # Criando agora uma função para automatizar os embeddings
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [29]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [38]:
def gerar_e_buscar_consulta(consulta, base, model):

  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), emdedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [39]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)



NameError: name 'emdedding_da_consulta' is not defined